<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Blender_Renderer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt_text](https://raw.githubusercontent.com/qsardor/GoogleColabProjects/refs/heads/main/googlecolabimage.png)

# 🎬 **Render Blender files with Google Colab**
> **Last updated:** 01.10.2025

* **Blender versions up to 4.5.3** are available! 📚
* You can also add your own version of Blender from the [Blender Repository](https://ftp.nluug.nl/pub/graphics/blender/release). 👷

## **📝 Notes:**
* Run the cells one by one, from top to bottom. ▶️
* To copy a file's path, use the file explorer on the left, click the three dots (⋮) next to the file, and select `Copy path`. 🗄️
* Please avoid spaces in filenames (e.g., rename `My Scene.blend` to `My_Scene.blend`). 📝
* If you upload your `.blend` file directly, please wait for the upload to complete before running the cells. 🔽
* **⚠️ Free Tier Warning:** Google Colab's free sessions have strict time limits, which can be as short as 2 hours when using a GPU. For long animation sequences, it is **highly recommended** to use the `gdrive_output` option to save your renders directly to Google Drive and prevent data loss if the session disconnects.

## **📖 How to Use This Notebook (A Full Guide)**

### **Step 1: 📂 Prepare & Upload Your Files**
1.  **Crucial Step (Textures):** To avoid missing textures (which show up as bright purple), you must pack all external files into your project. In Blender, go to **File → External Data → Pack Resources**.
2.  **Crucial Step (Add-ons):** As a best practice, try to apply or remove any add-on-specific modifiers from your models before uploading to avoid dependencies.
3.  Upload your packed `.blend` file. **It is strongly recommended to rename your file to `File.blend`** (especially if it contains spaces) and place it in the main `/content/` directory. The default file path in the settings below is already set for this common use case.

### **Step 2: ☁️ Connect to Google Drive (Optional)**
* Run the **first code cell** (`☁️ 1. Mount Google Drive`) if you want to use Google Drive for your `.blend` file, render outputs, or add-ons.

### **Step 3: ⚙️ Configure Your Render Job (Detailed Explanation)**
This is the main control panel in the **second code cell** (`⚙️ 2. Configure Your Render Job`).

#### **Blender & Add-on Options**
* **`blender_version`**, **`project_name`**, **`blend_file_path`**: The basics for your project setup.
* **`install_addons`**: `✅ Checked` will install all `.zip` files from your `add-ons` folder (either in Google Drive or locally).

#### **Google Drive & Hardware Options**
* **`gdrive_output`**: `✅ Checked` saves renders and logs to `MyDrive/BlenderColab/[Project Name]/rendered/`.
* **`organize_gdrive`**: `✅ Checked` moves your source `.blend` file into the organized project folder on Drive.
* **`gpu_enabled`** & **`optix_enabled`**: Keep these checked for the fastest rendering performance, primarily for `CYCLES`.

#### **Render Settings Source**
* **`Use Settings from .blend File` (Default):** The simplest option. It uses the render engine, resolution, samples, and frame range saved inside your project file.
* **`Use Manual Overrides`:** Activates the section below, allowing you to control the render settings directly from this notebook.

#### **Manual Overrides**
> **Important:** This entire section does nothing unless you select `Use Manual Overrides` above.
* **`render_engine`**: Choose your render engine. `CYCLES` is a physically-based path tracer for high-quality, realistic renders. `EEVEE` is a real-time rasterization engine for much faster renders. All Blender versions in the list support both.
* **`use_denoising`**: `✅ Checked` enables Blender's built-in denoiser. **This only applies to the `CYCLES` engine.**
* **`render_animation_sequence`**: `✅ Checked` renders a sequence from `start_frame` to `end_frame`. `🔲 Unchecked` renders only the single frame specified in `start_frame`.
* **`width`**, **`height`**, **`samples`**, etc.: These let you set the resolution, quality, and frame range for your render.
* **`output_format`**: Choose the file format. Use `PNG` or `JPEG` for still images, and `FFMPEG (Video Render)` to output a video file directly.

### **Step 4: 🔍 Review Settings & Prepare to Render**
* Run the **third code cell** (`🔍 3. Review Settings...`). This cell does all the setup and prints a summary of your render job. **It does not start the render.**

### **Step 5: 🚀 START THE RENDER!**
* After you have reviewed the summary and are sure everything is correct, run the **fourth code cell** (`🚀 4. START THE RENDER!`) to begin.

### **Step 6: 📦 Download Your Renders (If not using Google Drive)**
* After the render finishes, if you didn't save to Google Drive, run the **fifth code cell** (`📦 5. Zip and Download Renders`) to save your work.

In [ ]:
#@title ☁️ 1. Mount Google Drive (Optional)
#@markdown Run this cell if your files are in Google Drive or you want to save your renders there.
from google.colab import drive

try:
  drive.mount("/content/drive", force_remount=True)
  print("✅ Google Drive mounted successfully!")
except Exception as e:
  print(f"⚠️ Could not mount Google Drive: {e}")

In [ ]:
#@title ⚙️ 2. Configure Your Render Job

#@markdown ###  Blender Options
blender_version = '4.5.3' #@param ['3.0.1','3.1.2','3.2.2','3.3.8','3.4.1','3.5.1','3.6.0','4.0.2','4.1.1','4.2.0','4.3.0','4.4.1','4.5.3'] {allow-input: true}
project_name = 'MyBlenderProject' #@param {type: 'string'}
blend_file_path = '/content/File.blend' #@param {type: 'string'}

#@markdown ### 🧩 Add-on Options
#@markdown Check this box to install all .zip files from the `add-ons` folder.
install_addons = False #@param {type:"boolean"}

#@markdown ### ☁️ Google Drive Options
gdrive_output = False #@param {type:"boolean"}
organize_gdrive = False #@param {type:"boolean"}

#@markdown ### 💻 Hardware Options
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### ⚙️ Render Settings Source
#@markdown Choose whether to use the settings saved in your `.blend` file or the manual overrides below.
render_settings_source = 'Use Settings from .blend File' #@param ["Use Settings from .blend File", "Use Manual Overrides"]

#@markdown ### ✨ Manual Overrides (only apply if 'Use Manual Overrides' is selected)
#@markdown > These settings are **ignored** unless you change the option above.
#@markdown > **To restore defaults, just copy the values below.**
#@markdown > `render_engine: CYCLES`, `use_denoising: True`, `render_animation_sequence: True`, `width: 1920`, `height: 1080`, `samples: 256`, `start_frame: 1`, `end_frame: 100`, `output_format: PNG`
render_engine = 'CYCLES' #@param ["CYCLES", "EEVEE"]
use_denoising = True #@param {type:"boolean"}
render_animation_sequence = True #@param {type:"boolean"}
#@markdown &nbsp;
width = 1920 #@param {type: 'integer'}
height = 1080 #@param {type: 'integer'}
samples = 256 #@param {type: 'integer'}
start_frame = 1 #@param {type: 'integer'} # If rendering a single frame, this is the frame that will be rendered.
end_frame = 100 #@param {type: 'integer'}
output_format = 'PNG' #@param ['PNG', 'OPEN_EXR', 'JPEG', 'TIFF', 'FFMPEG (Video Render)']

In [ ]:
#@title 🔍 3. Review Settings & Prepare to Render
#@markdown This cell prepares everything and shows you a summary of your settings. **It does not start the render.**

import os
import sys
from pathlib import Path

# --- Make render_cmd a global variable ---
render_cmd = ""

# --- Helper Functions ---
def create_project_dirs():
    os.makedirs(f"/content/{project_name}/rendered", exist_ok=True)
    os.makedirs(f"/content/{project_name}/blend", exist_ok=True)
    Path('/content/sample_data').exists() and os.system('rm -r /content/sample_data')

def setup_gdrive_dirs():
    if gdrive_output:
        os.makedirs(f"/content/drive/MyDrive/BlenderColab/{project_name}/rendered", exist_ok=True)
    if organize_gdrive and "/content/drive/MyDrive" in blend_file_path:
        os.makedirs(f"/content/drive/MyDrive/BlenderColab/{project_name}/blend", exist_ok=True)
        os.system(f'mv "{blend_file_path}" "/content/drive/MyDrive/BlenderColab/{project_name}/blend/"')
        print("Moved .blend file to organized GDrive folder.")

def check_and_copy_blend_file():
    print("🔍 Checking for .blend file...")
    blend_path = Path(blend_file_path)
    if not blend_path.exists():
        sys.exit(f"❌ ERROR: Blend file not found at: {blend_file_path}")
    try:
        with open(blend_path, 'rb') as f:
            header = f.read(12)
            if not header.startswith(b'BLENDER'):
                sys.exit(f"❌ ERROR: The file at {blend_file_path} is not a valid .blend file.")
            version_str = header[9:12].decode('ascii')
            blender_file_version = f"{version_str[0]}.{version_str[1:]}"
            print(f"✅ Blend file found and valid. Saved with Blender version: {blender_file_version}")
    except Exception as e:
        sys.exit(f"❌ ERROR: Could not read the .blend file. Details: {e}")
    print(f"   ↳ Copying '{os.path.basename(blend_file_path)}' to the project folder...")
    os.system(f'cp "{blend_file_path}" "/content/{project_name}/blend/"')

def get_gpu_info():
    global optix_enabled
    print("\n💻 Checking GPU info...")
    gpu_name = os.popen('nvidia-smi --query-gpu=gpu_name --format=csv,noheader').read().strip()
    print(f"✅ Current GPU: {gpu_name}")
    if "Tesla K80" in gpu_name and optix_enabled:
        print(f"⚠️ GPU does not support OptiX. Disabling OptiX.")
        optix_enabled = False

def download_blender():
    blender_path = Path(f"/content/{blender_version}")
    if blender_path.exists():
        print(f"\n📚 Blender {blender_version} already downloaded.")
        return
    blender_url_dict = {
        '3.0.1': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.0/blender-3.0.1-linux-x64.tar.xz", '3.1.2': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.1/blender-3.1.2-linux-x64.tar.xz", '3.2.2': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.2/blender-3.2.2-linux-x64.tar.xz", '3.3.8': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.3/blender-3.3.8-linux-x64.tar.xz", '3.4.1': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.4/blender-3.4.1-linux-x64.tar.xz", '3.5.1': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.5/blender-3.5.1-linux-x64.tar.xz", '3.6.0': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.6/blender-3.6.0-linux-x64.tar.xz", '4.0.2': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.0/blender-4.0.2-linux-x64.tar.xz", '4.1.1': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.1/blender-4.1.1-linux-x64.tar.xz", '4.2.0': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.2/blender-4.2.0-linux-x64.tar.xz", '4.3.0': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.3/blender-4.3.0-linux-x64.tar.xz", '4.4.1': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.4/blender-4.4.1-linux-x64.tar.xz", '4.5.3': "https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.5/blender-4.5.3-linux-x64.tar.xz"
    }
    print(f"\n🔽 Downloading Blender {blender_version}...")
    blender_url = blender_url_dict.get(blender_version, f"https://ftp.nluug.nl/pub/graphics/blender/release/Blender{blender_version.rpartition('.')[0]}/blender-{blender_version}-linux-x64.tar.xz")
    archive_name = os.path.basename(blender_url)
    os.system(f"wget -q --show-progress {blender_url}")
    print("   ↳ Extracting...")
    os.makedirs(blender_path)
    os.system(f"tar -xkf {archive_name} -C {blender_path} --strip-components=1")
    os.system(f"rm {archive_name}")
    print("✅ Blender setup complete.")

def create_gpu_script():
    script_content = f"""import re\nimport bpy\nscene = bpy.context.scene\nscene.cycles.device = 'GPU'\nprefs = bpy.context.preferences\nprefs.addons['cycles'].preferences.get_devices()\ncprefs = prefs.addons['cycles'].preferences\nfor compute_device_type in ('OPTIX', 'CUDA', 'HIP', 'METAL', 'ONEAPI', 'OPENCL', 'NONE'):\n    try:\n        cprefs.compute_device_type = compute_device_type\n        break\n    except TypeError:\n        pass\nfor device in cprefs.devices:\n    if not re.match('intel', device.name, re.I):\n        device.use = {gpu_enabled}\n    else:\n        device.use = False\n"""
    with open('setgpu.py', 'w') as f:
        f.write(script_content)

def build_and_summarize_command():
    global render_cmd
    blend_file_name = os.path.basename(blend_file_path)
    local_blend_path = f'/content/{project_name}/blend/{blend_file_name}'
    output_dir = f'/content/drive/MyDrive/BlenderColab/{project_name}/rendered/' if gdrive_output else f'/content/{project_name}/rendered/'
    output_path = f'{output_dir}{project_name}_####'

    # Determine engine from manual overrides if selected
    final_render_engine = render_engine if render_settings_source == 'Use Manual Overrides' else 'CYCLES' # Default to CYCLES if using file settings

    command = f'./blender -b "{local_blend_path}" -noaudio -E {final_render_engine}'

    # CYCLES-specific settings
    if final_render_engine == 'CYCLES':
      command += f' -P setgpu.py -- --cycles-device {"OPTIX" if optix_enabled else "CUDA"}'

    command += f' -o "{output_path}"'
    py_overrides = []
    summary = []

    if install_addons:
        gdrive_addon_dir = f'/content/drive/MyDrive/BlenderColab/add-ons'
        local_addon_dir = '/content/add-ons'
        os.makedirs(local_addon_dir, exist_ok=True)
        addon_dir_to_use = None
        if os.path.exists(gdrive_addon_dir) and os.listdir(gdrive_addon_dir):
            addon_dir_to_use = gdrive_addon_dir
        elif os.path.exists(local_addon_dir) and os.listdir(local_addon_dir):
             addon_dir_to_use = local_addon_dir
        if addon_dir_to_use:
            zip_files = [f for f in os.listdir(addon_dir_to_use) if f.endswith('.zip')]
            if zip_files:
                summary.append(f"Installing {len(zip_files)} add-on(s): {', '.join(zip_files)}")
                py_overrides.append('import bpy')
                for addon_zip in zip_files:
                    addon_path = os.path.join(addon_dir_to_use, addon_zip)
                    addon_module_name = addon_zip.replace('.zip', '')
                    py_overrides.append(f'bpy.ops.preferences.addon_install(overwrite=True, filepath=\"{addon_path}\")')
                    py_overrides.append(f'bpy.ops.preferences.addon_enable(module=\"{addon_module_name}\")')
    if render_settings_source == 'Use Manual Overrides':
        summary.append("Using Manual Overrides")
        summary.append(f"Render Engine: {render_engine}")
        summary.append(f"Resolution: {width}x{height}")
        summary.append(f"Samples: {samples}")
        if render_engine == 'CYCLES':
          summary.append(f"Denoising: {'On' if use_denoising else 'Off'}")
          py_overrides.append(f'bpy.context.scene.cycles.use_denoising = {use_denoising}')
        if render_animation_sequence:
            command += f' -s {start_frame} -e {end_frame} -a'
            summary.append(f"Mode: Animation | Frames: {start_frame} to {end_frame}")
        else:
            command += f' -f {start_frame}'
            summary.append(f"Mode: Single Frame | Frame: {start_frame}")
        api_output_format = 'FFMPEG' if 'FFMPEG' in output_format else output_format
        py_overrides.append(f'bpy.context.scene.render.image_settings.file_format = \"{api_output_format}\" ')
        if width > 0 and height > 0:
            py_overrides.append(f'bpy.context.scene.render.resolution_x = {width}')
            py_overrides.append(f'bpy.context.scene.render.resolution_y = {height}')
        if samples > 0 and render_engine == 'CYCLES':
            py_overrides.append(f'bpy.context.scene.cycles.samples = {samples}')
        elif samples > 0 and render_engine == 'EEVEE':
            py_overrides.append(f'bpy.context.scene.eevee.taa_render_samples = {samples}')

    else:
        command += ' -a'
        summary.append("Using settings from .blend file")
    if py_overrides:
        command += f' --python-expr "{'; '.join(py_overrides)}"'
    render_cmd = command
    print("\n" + "-"*50)
    print("          Render Job Summary")
    print("-"*50)
    print(f"  Project Name: {project_name}")
    print(f"  Blender Version: {blender_version}")
    print(f"  Output Location: {output_dir}")
    for item in summary:
        print(f"  - {item}")
    print("-"*50)
    print("\n⚠️  Please review the settings above carefully.")
    print("   If everything is correct, run the next cell to start the render.")
    print("-"*50)

# --- Main Execution ---
create_project_dirs()
if gdrive_output or organize_gdrive:
    setup_gdrive_dirs()
check_and_copy_blend_file()
get_gpu_info()
download_blender()
create_gpu_script()
build_and_summarize_command()


In [ ]:
#@title 🚀 4. START THE RENDER!
#@markdown Run this cell **after** reviewing the summary above to start the render process.
import datetime
import os

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"render_log_{timestamp}.txt"
output_location = f'/content/drive/MyDrive/BlenderColab/{project_name}/rendered/' if gdrive_output else f'/content/{project_name}/rendered/'
log_filepath = os.path.join(output_location, log_filename)

print("\n" + "-"*80)
print("🚀 Starting Blender Render...")
print(f"   Logs will be saved to: {log_filename} in your output folder.")
print("   Please wait. Real-time progress is not displayed here.")
print("-"*80 + "\n")

os.chdir(f'/content/{blender_version}')
os.system(f'{render_cmd} > "{log_filepath}" 2>&1')

print("\n" + "-"*80)
print("✅ Render Finished!")
print(f"   Check your output files in: {output_location}")
print(f"   Log file saved: {log_filename}")
print("-"*80 + "\n")

In [ ]:
#@title 📦 5. Zip and Download Renders (Optional)
#@markdown Run this if you didn't use Google Drive and want to download your renders and logs.

import os
from google.colab import files

local_render_path = f'/content/{project_name}/rendered'
zip_path = f'/content/{project_name}_render_output.zip'

if os.path.exists(local_render_path) and any(os.scandir(local_render_path)):
    print("Zipping files...")
    os.system(f'zip -r -j "{zip_path}" "{local_render_path}"')
    print(f'✅ Created zip file at: {zip_path}')
    files.download(zip_path)
else:
    print("🤔 Render folder not found or is empty. Did the render complete successfully?")


### ❤️ **Credits**
* **Author:** QSARDOR ([✈️ Telegram](https://t.me/qsardorblog) | [💻 GitHub](https://github.com/qsardor/))
* **More Cool Stuff:** [Google Colab Projects Repo](https://github.com/qsardor/GoogleColabProjects)